In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
# import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vp
import model_validation.ciff_sam_results as csr
import model_validation.ciff_sam_plots as csp

%load_ext autoreload
%autoreload 2

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch
ndbs
Fri Oct 15 19:52:01 PDT 2021


# Load data and set global index columns

In [2]:
results = csr.VivariumResults.cleaned_from_model_spec('4.5.2')
results.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

In [3]:
vp.set_global_index_columns(vp.INDEX_COLUMNS+['x_factor_effect'])
vp.INDEX_COLUMNS

['input_draw', 'scenario', 'x_factor_effect']

# Test new prevalence and transition rate functions with measure column added

In [5]:
wasting_state_prevalence_by_age_year = csr.get_prevalence(results, 'wasting_state', ['age', 'year'])
wasting_state_prevalence_by_age_year

,age,year,input_draw,scenario,x_factor_effect,wasting_state,value,numerator_measure,denominator_measure,multiplier,measure
0,1-5_months,2022,29,baseline,1.1,mild_child_wasting,0.203855,state_person_time,person_time,1,prevalence
1,1-5_months,2022,29,baseline,1.1,moderate_acute_malnutrition,0.090002,state_person_time,person_time,1,prevalence
2,1-5_months,2022,29,baseline,1.1,severe_acute_malnutrition,0.030675,state_person_time,person_time,1,prevalence
3,1-5_months,2022,29,baseline,1.1,susceptible_to_child_wasting,0.675468,state_person_time,person_time,1,prevalence
...,...,...,...,...,...,...,...,...,...,...,...
21596,late_neonatal,2026,946,wasting_treatment,1.5,mild_child_wasting,0.200041,state_person_time,person_time,1,prevalence
21597,late_neonatal,2026,946,wasting_treatment,1.5,moderate_acute_malnutrition,0.089943,state_person_time,person_time,1,prevalence
21598,late_neonatal,2026,946,wasting_treatment,1.5,severe_acute_malnutrition,0.032869,state_person_time,person_time,1,prevalence
21599,late_neonatal,2026,946,wasting_treatment,1.5,susceptible_to_child_wasting,0.677147,state_person_time,person_time,1,prevalence


In [6]:
stunting_state_prevalence_by_age_year = csr.get_prevalence(results, 'stunting_state', ['age', 'year'])
stunting_state_prevalence_by_age_year

,age,year,input_draw,scenario,x_factor_effect,stunting_state,value,numerator_measure,denominator_measure,multiplier,measure
0,1-5_months,2022,29,baseline,1.1,cat1,0.057169,state_person_time,person_time,1,prevalence
1,1-5_months,2022,29,baseline,1.1,cat2,0.117429,state_person_time,person_time,1,prevalence
2,1-5_months,2022,29,baseline,1.1,cat3,0.201148,state_person_time,person_time,1,prevalence
3,1-5_months,2022,29,baseline,1.1,cat4,0.624255,state_person_time,person_time,1,prevalence
...,...,...,...,...,...,...,...,...,...,...,...
21596,late_neonatal,2026,946,wasting_treatment,1.5,cat1,0.000000,state_person_time,person_time,1,prevalence
21597,late_neonatal,2026,946,wasting_treatment,1.5,cat2,0.000000,state_person_time,person_time,1,prevalence
21598,late_neonatal,2026,946,wasting_treatment,1.5,cat3,0.000000,state_person_time,person_time,1,prevalence
21599,late_neonatal,2026,946,wasting_treatment,1.5,cat4,1.000000,state_person_time,person_time,1,prevalence


In [8]:
sq_lns_coverage_by_year_sex = csr.get_prevalence(results, 'sq_lns', ['year', 'sex'])
sq_lns_coverage_by_year_sex

,year,sex,input_draw,scenario,x_factor_effect,sq_lns,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,female,29,baseline,1.1,covered,0.0,person_time,person_time,1,prevalence
1,2022,female,29,baseline,1.1,uncovered,1.0,person_time,person_time,1,prevalence
2,2022,female,29,baseline,1.2,covered,0.0,person_time,person_time,1,prevalence
3,2022,female,29,baseline,1.2,uncovered,1.0,person_time,person_time,1,prevalence
...,...,...,...,...,...,...,...,...,...,...,...
3596,2026,male,946,wasting_treatment,1.4,covered,0.0,person_time,person_time,1,prevalence
3597,2026,male,946,wasting_treatment,1.4,uncovered,1.0,person_time,person_time,1,prevalence
3598,2026,male,946,wasting_treatment,1.5,covered,0.0,person_time,person_time,1,prevalence
3599,2026,male,946,wasting_treatment,1.5,uncovered,1.0,person_time,person_time,1,prevalence


In [9]:
cause_transition_rates_by_age_year_sex = csr.get_transition_rates(
    results, 'cause', ['age', 'year', 'sex'])
cause_transition_rates_by_age_year_sex

,age,year,sex,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier,measure
0,1-5_months,2022,female,diarrheal_diseases,29,baseline,1.1,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,64.509197,transition_count,state_person_time,1,transition_rate
1,1-5_months,2022,female,diarrheal_diseases,29,baseline,1.2,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,64.509197,transition_count,state_person_time,1,transition_rate
2,1-5_months,2022,female,diarrheal_diseases,29,baseline,1.3,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,64.509197,transition_count,state_person_time,1,transition_rate
3,1-5_months,2022,female,diarrheal_diseases,29,baseline,1.4,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,64.509197,transition_count,state_person_time,1,transition_rate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64796,late_neonatal,2026,male,susceptible_to_measles,946,wasting_treatment,1.2,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1,transition_rate
64797,late_neonatal,2026,male,susceptible_to_measles,946,wasting_treatment,1.3,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1,transition_rate
64798,late_neonatal,2026,male,susceptible_to_measles,946,wasting_treatment,1.4,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1,transition_rate
64799,late_neonatal,2026,male,susceptible_to_measles,946,wasting_treatment,1.5,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1,transition_rate


In [11]:
x_factor_wasting_rate_ratios_by_year = csr.get_x_factor_wasting_transition_rate_ratio(
    results, ['year']
)
x_factor_wasting_rate_ratios_by_year

,year,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_x_factor,denominator_x_factor,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.1,cat1,cat2,1.111603,transition_rate,transition_rate,1,rate_ratio
1,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.2,cat1,cat2,1.214612,transition_rate,transition_rate,1,rate_ratio
2,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.3,cat1,cat2,1.309192,transition_rate,transition_rate,1,rate_ratio
3,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.4,cat1,cat2,1.397571,transition_rate,transition_rate,1,rate_ratio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.2,cat1,cat2,1.172504,transition_rate,transition_rate,1,rate_ratio
6297,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.3,cat1,cat2,1.278598,transition_rate,transition_rate,1,rate_ratio
6298,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.4,cat1,cat2,1.379262,transition_rate,transition_rate,1,rate_ratio
6299,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.5,cat1,cat2,1.457853,transition_rate,transition_rate,1,rate_ratio


# Look at deaths dataframe

In [12]:
results.deaths

,sex,year,cause,measure,input_draw,scenario,x_factor_effect,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,1.1,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
259196,male,2026,severe_acute_malnutrition,death,946,wasting_treatment,1.5,0.0,susceptible_to_child_wasting,2_to_4
259197,male,2026,severe_acute_malnutrition,death,946,wasting_treatment,1.5,0.0,mild_child_wasting,2_to_4
259198,male,2026,severe_acute_malnutrition,death,946,wasting_treatment,1.5,0.0,moderate_acute_malnutrition,2_to_4
259199,male,2026,severe_acute_malnutrition,death,946,wasting_treatment,1.5,0.0,severe_acute_malnutrition,2_to_4


In [13]:
results.deaths.value.sum()

977732.0

In [24]:
results.ylls

,sex,year,cause,measure,input_draw,scenario,x_factor_effect,value,wasting_state,age
0,female,2022,diarrheal_diseases,ylls,29,baseline,1.1,0.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,ylls,29,baseline,1.1,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,ylls,29,baseline,1.1,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,ylls,29,baseline,1.1,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
259196,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,1.5,0.0,susceptible_to_child_wasting,2_to_4
259197,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,1.5,0.0,mild_child_wasting,2_to_4
259198,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,1.5,0.0,moderate_acute_malnutrition,2_to_4
259199,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,1.5,0.0,severe_acute_malnutrition,2_to_4


# Try out some different possible function signatures to see what feels more natural...

# Test f-strings with .query()

In [14]:
results.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,covered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,uncovered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...
345596,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,16475.270363,cat2,uncovered,covered,2_to_4
345597,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,3419.826146,cat1,uncovered,covered,2_to_4
345598,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,1704.976044,cat2,uncovered,uncovered,2_to_4
345599,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,384.646133,cat1,uncovered,uncovered,2_to_4


In [15]:
factor='x_factor'
reference_category = 'cat2'
results.wasting_state_person_time.query(f"{factor} == '{reference_category}'")

,sex,year,wasting_state,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,covered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,uncovered,early_neonatal
4,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,15.400411,cat2,uncovered,covered,early_neonatal
6,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,16.733744,cat2,uncovered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...
345592,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,0.000000,cat2,covered,covered,2_to_4
345594,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,0.000000,cat2,covered,uncovered,2_to_4
345596,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,16475.270363,cat2,uncovered,covered,2_to_4
345598,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,1704.976044,cat2,uncovered,uncovered,2_to_4


In [16]:
results.wasting_state_person_time.query(f"{factor} != '{reference_category}'")

,sex,year,wasting_state,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,covered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,uncovered,early_neonatal
5,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,3.596167,cat1,uncovered,covered,early_neonatal
7,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,4.053388,cat1,uncovered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...
345593,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,0.000000,cat1,covered,covered,2_to_4
345595,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,0.000000,cat1,covered,uncovered,2_to_4
345597,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,3419.826146,cat1,uncovered,covered,2_to_4
345599,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,384.646133,cat1,uncovered,uncovered,2_to_4


# Test new get_relative_risk function...

In [20]:
under_6mo, over_6mo, all_ages = map(list, csr.get_age_group_bins('6-11_months', 'all_ages'))
over_6mo

['6-11_months', '12_to_23_months', '2_to_4']

In [23]:
csr.get_relative_risk(
    data=results,
    measure='prevalence',
    outcome='wasting_state',
    strata=['year', 'age'],
    factor='sq_lns',
    reference_category='uncovered',
    prefilter_query = f"age in {over_6mo} and scenario=='sqlns'",
)

,year,age,wasting_state,input_draw,scenario,x_factor_effect,numerator_sq_lns,denominator_sq_lns,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.1,covered,uncovered,0.000000,prevalence,prevalence,1,relative_risk
1,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.2,covered,uncovered,0.000000,prevalence,prevalence,1,relative_risk
2,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.3,covered,uncovered,0.000000,prevalence,prevalence,1,relative_risk
3,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.4,covered,uncovered,0.000000,prevalence,prevalence,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.2,covered,uncovered,0.978525,prevalence,prevalence,1,relative_risk
3597,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.3,covered,uncovered,0.982514,prevalence,prevalence,1,relative_risk
3598,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.4,covered,uncovered,0.984282,prevalence,prevalence,1,relative_risk
3599,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.5,covered,uncovered,0.984119,prevalence,prevalence,1,relative_risk


In [27]:
csr.get_relative_risk(
    data=results,
    measure='prevalence',
    outcome='sq_lns',
    strata=['year', 'age'],
    factor='x_factor',
    reference_category='cat2',
    prefilter_query = f"age in {over_6mo} and scenario=='sqlns'",
)

,year,age,sq_lns,input_draw,scenario,x_factor_effect,numerator_x_factor,denominator_x_factor,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,12_to_23_months,covered,29,sqlns,1.1,cat1,cat2,NaN,prevalence,prevalence,1,relative_risk
1,2022,12_to_23_months,covered,29,sqlns,1.2,cat1,cat2,NaN,prevalence,prevalence,1,relative_risk
2,2022,12_to_23_months,covered,29,sqlns,1.3,cat1,cat2,NaN,prevalence,prevalence,1,relative_risk
3,2022,12_to_23_months,covered,29,sqlns,1.4,cat1,cat2,NaN,prevalence,prevalence,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,2026,6-11_months,uncovered,946,sqlns,1.2,cat1,cat2,0.960908,prevalence,prevalence,1,relative_risk
1797,2026,6-11_months,uncovered,946,sqlns,1.3,cat1,cat2,0.960908,prevalence,prevalence,1,relative_risk
1798,2026,6-11_months,uncovered,946,sqlns,1.4,cat1,cat2,0.960908,prevalence,prevalence,1,relative_risk
1799,2026,6-11_months,uncovered,946,sqlns,1.5,cat1,cat2,0.960758,prevalence,prevalence,1,relative_risk


## See if I'm able to compute cause transitions stratified by wasting state

Yes. Therefore I should be able to compute the relative risk of cause transitions with respect to wasting state.

In [57]:
csr.get_transition_rates(results, 'cause', ['year', 'wasting_state'])

,year,wasting_state,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,mild_child_wasting,diarrheal_diseases,29,baseline,1.1,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,63.340543,transition_count,state_person_time,1,transition_rate
1,2022,mild_child_wasting,diarrheal_diseases,29,baseline,1.2,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,63.255001,transition_count,state_person_time,1,transition_rate
2,2022,mild_child_wasting,diarrheal_diseases,29,baseline,1.3,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,63.393561,transition_count,state_person_time,1,transition_rate
3,2022,mild_child_wasting,diarrheal_diseases,29,baseline,1.4,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,63.424634,transition_count,state_person_time,1,transition_rate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21596,2026,susceptible_to_child_wasting,susceptible_to_measles,946,wasting_treatment,1.2,susceptible_to_measles_to_measles,measles,0.004698,transition_count,state_person_time,1,transition_rate
21597,2026,susceptible_to_child_wasting,susceptible_to_measles,946,wasting_treatment,1.3,susceptible_to_measles_to_measles,measles,0.004705,transition_count,state_person_time,1,transition_rate
21598,2026,susceptible_to_child_wasting,susceptible_to_measles,946,wasting_treatment,1.4,susceptible_to_measles_to_measles,measles,0.004757,transition_count,state_person_time,1,transition_rate
21599,2026,susceptible_to_child_wasting,susceptible_to_measles,946,wasting_treatment,1.5,susceptible_to_measles_to_measles,measles,0.004865,transition_count,state_person_time,1,transition_rate


In [59]:
%%time
wasting_cause_transition_rr = csr.get_relative_risk(
    results,
    measure='transition_rate',
    outcome='cause',
    strata=['year', 'age'],
    factor='wasting_state',
    reference_category='susceptible_to_child_wasting'
)
wasting_cause_transition_rr

CPU times: user 1.5 s, sys: 955 µs, total: 1.5 s
Wall time: 1.51 s


,year,age,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_wasting_state,denominator_wasting_state,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.1,mild_child_wasting,susceptible_to_child_wasting,1.015873,transition_rate,transition_rate,1,relative_risk
1,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.1,moderate_acute_malnutrition,susceptible_to_child_wasting,1.003066,transition_rate,transition_rate,1,relative_risk
2,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.1,severe_acute_malnutrition,susceptible_to_child_wasting,1.005805,transition_rate,transition_rate,1,relative_risk
3,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.2,mild_child_wasting,susceptible_to_child_wasting,1.015873,transition_rate,transition_rate,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97196,2026,late_neonatal,susceptible_to_measles_to_measles,susceptible_to_measles,measles,946,wasting_treatment,1.4,severe_acute_malnutrition,susceptible_to_child_wasting,NaN,transition_rate,transition_rate,1,relative_risk
97197,2026,late_neonatal,susceptible_to_measles_to_measles,susceptible_to_measles,measles,946,wasting_treatment,1.5,mild_child_wasting,susceptible_to_child_wasting,NaN,transition_rate,transition_rate,1,relative_risk
97198,2026,late_neonatal,susceptible_to_measles_to_measles,susceptible_to_measles,measles,946,wasting_treatment,1.5,moderate_acute_malnutrition,susceptible_to_child_wasting,NaN,transition_rate,transition_rate,1,relative_risk
97199,2026,late_neonatal,susceptible_to_measles_to_measles,susceptible_to_measles,measles,946,wasting_treatment,1.5,severe_acute_malnutrition,susceptible_to_child_wasting,NaN,transition_rate,transition_rate,1,relative_risk


In [60]:
wasting_cause_transition_rr.dropna()

,year,age,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_wasting_state,denominator_wasting_state,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.1,mild_child_wasting,susceptible_to_child_wasting,1.015873,transition_rate,transition_rate,1,relative_risk
1,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.1,moderate_acute_malnutrition,susceptible_to_child_wasting,1.003066,transition_rate,transition_rate,1,relative_risk
2,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.1,severe_acute_malnutrition,susceptible_to_child_wasting,1.005805,transition_rate,transition_rate,1,relative_risk
3,2022,1-5_months,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases,29,baseline,1.2,mild_child_wasting,susceptible_to_child_wasting,1.015873,transition_rate,transition_rate,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96656,2026,late_neonatal,susceptible_to_lower_respiratory_infections_to...,susceptible_to_lower_respiratory_infections,lower_respiratory_infections,946,wasting_treatment,1.4,severe_acute_malnutrition,susceptible_to_child_wasting,0.274406,transition_rate,transition_rate,1,relative_risk
96657,2026,late_neonatal,susceptible_to_lower_respiratory_infections_to...,susceptible_to_lower_respiratory_infections,lower_respiratory_infections,946,wasting_treatment,1.5,mild_child_wasting,susceptible_to_child_wasting,1.083203,transition_rate,transition_rate,1,relative_risk
96658,2026,late_neonatal,susceptible_to_lower_respiratory_infections_to...,susceptible_to_lower_respiratory_infections,lower_respiratory_infections,946,wasting_treatment,1.5,moderate_acute_malnutrition,susceptible_to_child_wasting,1.306290,transition_rate,transition_rate,1,relative_risk
96659,2026,late_neonatal,susceptible_to_lower_respiratory_infections_to...,susceptible_to_lower_respiratory_infections,lower_respiratory_infections,946,wasting_treatment,1.5,severe_acute_malnutrition,susceptible_to_child_wasting,0.274406,transition_rate,transition_rate,1,relative_risk


In [61]:
wasting_cause_transition_rr.query("transition == 'susceptible_to_diarrheal_diseases_to_diarrheal_diseases'")

,year,age,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_wasting_state,denominator_wasting_state,value,numerator_measure,denominator_measure,multiplier,measure
1620,2022,1-5_months,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,29,baseline,1.1,mild_child_wasting,susceptible_to_child_wasting,1.212666,transition_rate,transition_rate,1,relative_risk
1621,2022,1-5_months,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,29,baseline,1.1,moderate_acute_malnutrition,susceptible_to_child_wasting,2.665525,transition_rate,transition_rate,1,relative_risk
1622,2022,1-5_months,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,29,baseline,1.1,severe_acute_malnutrition,susceptible_to_child_wasting,9.484004,transition_rate,transition_rate,1,relative_risk
1623,2022,1-5_months,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,29,baseline,1.2,mild_child_wasting,susceptible_to_child_wasting,1.212666,transition_rate,transition_rate,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96116,2026,late_neonatal,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,946,wasting_treatment,1.4,severe_acute_malnutrition,susceptible_to_child_wasting,1.074340,transition_rate,transition_rate,1,relative_risk
96117,2026,late_neonatal,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,946,wasting_treatment,1.5,mild_child_wasting,susceptible_to_child_wasting,0.978254,transition_rate,transition_rate,1,relative_risk
96118,2026,late_neonatal,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,946,wasting_treatment,1.5,moderate_acute_malnutrition,susceptible_to_child_wasting,1.166195,transition_rate,transition_rate,1,relative_risk
96119,2026,late_neonatal,susceptible_to_diarrheal_diseases_to_diarrheal...,susceptible_to_diarrheal_diseases,diarrheal_diseases,946,wasting_treatment,1.5,severe_acute_malnutrition,susceptible_to_child_wasting,1.074340,transition_rate,transition_rate,1,relative_risk


# Compare `get_relative_risk` with the older `get_x_factor_wasting_transition_rate_ratio`

They give the same results. Woo hoo!

In [35]:
x_factor_wasting_rr1 = csr.get_relative_risk(
    data=results,
    measure='transition_rate',
    outcome='wasting',
    strata='year',
    factor='x_factor',
    reference_category='cat2',
    prefilter_query=f"age in {over_6mo}"
)
x_factor_wasting_rr1

,year,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_x_factor,denominator_x_factor,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.1,cat1,cat2,1.111603,transition_rate,transition_rate,1,relative_risk
1,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.2,cat1,cat2,1.214612,transition_rate,transition_rate,1,relative_risk
2,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.3,cat1,cat2,1.309192,transition_rate,transition_rate,1,relative_risk
3,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.4,cat1,cat2,1.397571,transition_rate,transition_rate,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.2,cat1,cat2,1.172504,transition_rate,transition_rate,1,relative_risk
6297,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.3,cat1,cat2,1.278598,transition_rate,transition_rate,1,relative_risk
6298,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.4,cat1,cat2,1.379262,transition_rate,transition_rate,1,relative_risk
6299,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.5,cat1,cat2,1.457853,transition_rate,transition_rate,1,relative_risk


In [36]:
x_factor_wasting_rr2 = csr.get_x_factor_wasting_transition_rate_ratio(results, 'year')
x_factor_wasting_rr2

,year,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_x_factor,denominator_x_factor,value,numerator_measure,denominator_measure,multiplier,measure
0,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.1,cat1,cat2,1.111603,transition_rate,transition_rate,1,rate_ratio
1,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.2,cat1,cat2,1.214612,transition_rate,transition_rate,1,rate_ratio
2,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.3,cat1,cat2,1.309192,transition_rate,transition_rate,1,rate_ratio
3,2022,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,baseline,1.4,cat1,cat2,1.397571,transition_rate,transition_rate,1,rate_ratio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.2,cat1,cat2,1.172504,transition_rate,transition_rate,1,rate_ratio
6297,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.3,cat1,cat2,1.278598,transition_rate,transition_rate,1,rate_ratio
6298,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.4,cat1,cat2,1.379262,transition_rate,transition_rate,1,rate_ratio
6299,2026,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting,946,wasting_treatment,1.5,cat1,cat2,1.457853,transition_rate,transition_rate,1,rate_ratio


In [39]:
vp.value(x_factor_wasting_rr1, include=['year', 'transition']).equals(
    vp.value(x_factor_wasting_rr2, include=['year', 'transition'])
)

True

# Compare `get_relative_risk` with older `get_sqlns_risk_prevalence_ratio` function

I can't right now because I changed some code...

In [41]:
sqlns_stunting_rr1 = csr.get_relative_risk(
    results,
    measure='prevalence',
    outcome='stunting_state',
    strata=['year', 'sex', 'age'],
    factor='sq_lns',
    reference_category='uncovered',
    prefilter_query=f"age in {over_6mo} and scenario=='sqlns' and year>'2022'"
)
sqlns_stunting_rr1

,year,sex,age,stunting_state,input_draw,scenario,x_factor_effect,numerator_sq_lns,denominator_sq_lns,value,numerator_measure,denominator_measure,multiplier,measure
0,2023,female,12_to_23_months,cat1,29,sqlns,1.1,covered,uncovered,0.840835,prevalence,prevalence,1,relative_risk
1,2023,female,12_to_23_months,cat1,29,sqlns,1.2,covered,uncovered,0.840881,prevalence,prevalence,1,relative_risk
2,2023,female,12_to_23_months,cat1,29,sqlns,1.3,covered,uncovered,0.840911,prevalence,prevalence,1,relative_risk
3,2023,female,12_to_23_months,cat1,29,sqlns,1.4,covered,uncovered,0.841058,prevalence,prevalence,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5756,2026,male,6-11_months,cat4,946,sqlns,1.2,covered,uncovered,0.966064,prevalence,prevalence,1,relative_risk
5757,2026,male,6-11_months,cat4,946,sqlns,1.3,covered,uncovered,0.966064,prevalence,prevalence,1,relative_risk
5758,2026,male,6-11_months,cat4,946,sqlns,1.4,covered,uncovered,0.966064,prevalence,prevalence,1,relative_risk
5759,2026,male,6-11_months,cat4,946,sqlns,1.5,covered,uncovered,0.965989,prevalence,prevalence,1,relative_risk


# Compare `get_relative_risk` with older `get_sqlns_mam_incidence_ratio` function

They're equal. Woo hoo!

In [50]:
%%time
sqlns_mam_incidence_rr1 = csr.get_relative_risk(
    results,
    measure='transition_rate',
    outcome='wasting',
    strata=['year', 'sex', 'age'],
    factor='sq_lns',
    reference_category='uncovered',
    prefilter_query=f"age in {over_6mo} and scenario=='sqlns' and year>'2022'"
).query(
    "transition=='mild_child_wasting_to_moderate_acute_malnutrition'"
)
sqlns_mam_incidence_rr1

CPU times: user 536 ms, sys: 13 µs, total: 536 ms
Wall time: 536 ms


,year,sex,age,transition,from_state,to_state,input_draw,scenario,x_factor_effect,numerator_sq_lns,denominator_sq_lns,value,numerator_measure,denominator_measure,multiplier,measure
0,2023,female,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,sqlns,1.1,covered,uncovered,0.737159,transition_rate,transition_rate,1,relative_risk
1,2023,female,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,sqlns,1.2,covered,uncovered,0.742849,transition_rate,transition_rate,1,relative_risk
2,2023,female,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,sqlns,1.3,covered,uncovered,0.750575,transition_rate,transition_rate,1,relative_risk
3,2023,female,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,29,sqlns,1.4,covered,uncovered,0.740593,transition_rate,transition_rate,1,relative_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9716,2026,male,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,946,sqlns,1.2,covered,uncovered,0.795206,transition_rate,transition_rate,1,relative_risk
9717,2026,male,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,946,sqlns,1.3,covered,uncovered,0.799953,transition_rate,transition_rate,1,relative_risk
9718,2026,male,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,946,sqlns,1.4,covered,uncovered,0.816933,transition_rate,transition_rate,1,relative_risk
9719,2026,male,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition,946,sqlns,1.5,covered,uncovered,0.831856,transition_rate,transition_rate,1,relative_risk


In [49]:
%%time
sqlns_mam_incidence_rr2 = csr.get_sqlns_mam_incidence_ratio(results)
sqlns_mam_incidence_rr2

CPU times: user 281 ms, sys: 6.35 ms, total: 287 ms
Wall time: 294 ms


,year,sex,age,input_draw,scenario,x_factor_effect,value,numerator_measure,denominator_measure,multiplier
0,2023,female,12_to_23_months,29,sqlns,1.1,0.737159,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
1,2023,female,12_to_23_months,29,sqlns,1.2,0.742849,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
2,2023,female,12_to_23_months,29,sqlns,1.3,0.750575,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
3,2023,female,12_to_23_months,29,sqlns,1.4,0.740593,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
...,...,...,...,...,...,...,...,...,...,...
1436,2026,male,6-11_months,946,sqlns,1.2,0.795206,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
1437,2026,male,6-11_months,946,sqlns,1.3,0.799953,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
1438,2026,male,6-11_months,946,sqlns,1.4,0.816933,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1
1439,2026,male,6-11_months,946,sqlns,1.5,0.831856,mam_incidence_rate_among_sqlns_covered,mam_incidence_rate_among_sqlns_uncovered,1


In [52]:
vp.value(sqlns_mam_incidence_rr1, include=['year', 'sex', 'age']).equals(
    vp.value(sqlns_mam_incidence_rr2, include=['year', 'sex', 'age'])
)

True

# I can't use wasting state as a risk factor for `get_relative_risk` because the person-time table isn't stratified by wasting state...

# Here's the prevalence calculation I need for the above relative risk:

# Compute prevalence of SQLNS conditional on year, age, and wasting state

Currently my `get_prevalence` function can't do this because it uses the `person_time` table in the denominator, not the `wasting_state_person_time` table.

A solution would be to write a function to search through available person-time and state-person-time tables for one that has the needed stratification, and call that function from within `get_prevalence` or other functions that need person-time.

In [30]:
prefilter_query = f"age in {over_6mo} and scenario=='sqlns'"
vp.ratio(
    results.wasting_state_person_time.query(prefilter_query),
    results.wasting_state_person_time.query(prefilter_query),
    strata=['year', 'age', 'wasting_state'],
    numerator_broadcast='sq_lns',
)

,year,age,wasting_state,input_draw,scenario,x_factor_effect,sq_lns,value,numerator_measure,denominator_measure,multiplier
0,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.1,covered,0.000000,state_person_time,state_person_time,1
1,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.1,uncovered,1.000000,state_person_time,state_person_time,1
2,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.2,covered,0.000000,state_person_time,state_person_time,1
3,2022,12_to_23_months,mild_child_wasting,29,sqlns,1.2,uncovered,1.000000,state_person_time,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...
7196,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.4,covered,0.895991,state_person_time,state_person_time,1
7197,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.4,uncovered,0.104009,state_person_time,state_person_time,1
7198,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.5,covered,0.895979,state_person_time,state_person_time,1
7199,2026,6-11_months,susceptible_to_child_wasting,946,sqlns,1.5,uncovered,0.104021,state_person_time,state_person_time,1


In [62]:
sys.argv

['/ihme/homes/ndbs/miniconda3/envs/ciff-sam-38/lib/python3.8/site-packages/ipykernel_launcher.py',
 '-f',
 '/ihme/homes/ndbs/.local/share/jupyter/runtime/kernel-dec7c3d9-391c-425e-b1b2-a0f76cf58ac0.json']

In [63]:
type(sys.argv)

list